In [98]:
%pip install -U kaleido

In [99]:
%pip install dash pandas yfinance plotly reportlab jupyter-dash flask_caching kaleido

In [100]:
import jupyter_dash
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import requests
import json
import io
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use('Agg')

# Initialize JupyterDash app
app = dash.Dash(__name__, external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"])

In [101]:
def get_stocks_finnhub():
    """
    Get stocks by sector using Finnhub API with fallback
    """
    FINNHUB_KEY = "cut6te1r01qrsirl3cpgcut6te1r01qrsirl3cq0"  # Replace with your key
    base_url = "https://finnhub.io/api/v1"

    # Define sector mappings
    sector_mappings = {
        'Technology': ['Technology', 'Software', 'Semiconductors'],
        'Media & Communications': ['Media', 'Communication Services'],
        'Consumer': ['Retail', 'Consumer products', 'Consumer Services'],
        'Financial': ['Banking', 'Financial Services', 'Insurance'],
        'Healthcare': ['Pharmaceuticals', 'Healthcare', 'Biotechnology'],
        'Industrial': ['Manufacturing', 'Industrial Services']
    }

    fallback_data = {
        'Technology': ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'AMD', 'INTC', 'TSM', 'AVGO', 'ADBE', 'CRM'],
        'Healthcare': ['JNJ', 'UNH', 'PFE', 'ABT', 'TMO', 'MRK', 'ABBV', 'DHR', 'BMY', 'AMGN'],
        'Financial': ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'BLK', 'SPGI', 'AXP', 'V'],
        'Consumer': ['AMZN', 'TSLA', 'HD', 'NKE', 'MCD', 'SBUX', 'TGT', 'LOW', 'F', 'GM'],
        'Media & Communications': ['META', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'TMUS', 'ATVI', 'EA', 'TTWO']
    }

    try:
        print("Testing Finnhub API connection...")
        test_response = requests.get(
            f"{base_url}/stock/symbol",
            params={"exchange": "US", "token": FINNHUB_KEY}
        )

        if test_response.status_code != 200:
            print("Using fallback data due to API connection issue")
            return fallback_data

        return fallback_data

    except Exception as e:
        print(f"Error in Finnhub processing: {str(e)}")
        return fallback_data

# Initialize sector data
SECTOR_STOCKS = get_stocks_finnhub()
SECTORS = list(SECTOR_STOCKS.keys())

Testing Finnhub API connection...


In [102]:
def get_stock_performance(sector, years_lookback):
    """
    Get stock performance data for a given sector
    """
    performance_data = []
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years_lookback*365)

    print(f"Fetching data for {sector} from {start_date} to {end_date}")

    stocks = SECTOR_STOCKS.get(sector, [])
    if not stocks:
        print(f"No stocks found for sector: {sector}")
        return pd.DataFrame()

    batch_size = 5
    for i in range(0, len(stocks), batch_size):
        batch = stocks[i:i+batch_size]

        try:
            print(f"Processing batch: {batch}")
            data = yf.download(
                batch,
                start=start_date,
                end=end_date,
                group_by='ticker',
                auto_adjust=True
            )

            for ticker in batch:
                try:
                    if isinstance(data, pd.DataFrame) and len(batch) == 1:
                        stock_data = data
                    else:
                        stock_data = data[ticker]

                    if not stock_data.empty:
                        initial_price = stock_data['Close'].iloc[0]
                        final_price = stock_data['Close'].iloc[-1]
                        growth = ((final_price - initial_price) / initial_price) * 100

                        try:
                            ticker_info = yf.Ticker(ticker)
                            company_name = ticker_info.info.get('longName', ticker)
                        except:
                            company_name = ticker

                        performance_data.append({
                            'Ticker': ticker,
                            'Company': company_name,
                            'Growth': growth,
                            'Initial Price': initial_price,
                            'Final Price': final_price
                        })
                        print(f"Processed {ticker}: Growth = {growth:.2f}%")
                except Exception as e:
                    print(f"Error processing {ticker}: {e}")
                    continue

        except Exception as e:
            print(f"Error downloading batch data: {e}")
            continue

    df = pd.DataFrame(performance_data)
    print(f"Successfully processed {len(df)} stocks for {sector}")
    return df

In [103]:
def generate_bar_chart(df, sector, years):
    """
    Generate bar chart using matplotlib
    """
    try:
        plt.figure(figsize=(10, 6))
        colors = plt.cm.Set3(np.linspace(0, 1, len(df)))

        bars = plt.barh(df['Ticker'], df['Growth'])

        # Color each bar
        for bar, color in zip(bars, colors):
            bar.set_color(color)

        plt.title(f'{sector} - Top Performers ({years} Year)')
        plt.xlabel('Growth (%)')
        plt.ylabel('Ticker')

        # Add value labels
        for i, v in enumerate(df['Growth']):
            plt.text(v, i, f' {v:.1f}%', va='center')

        plt.tight_layout()

        img_stream = io.BytesIO()
        plt.savefig(img_stream, format='png', bbox_inches='tight', dpi=300)
        img_stream.seek(0)
        plt.close()

        return Image(img_stream, width=9*inch, height=3*inch)
    except Exception as e:
        print(f"Error generating bar chart: {str(e)}")
        return None



In [104]:
def generate_sector_heatmap(sector_data, years):
    """
    Generate a heatmap using matplotlib/seaborn with stock tickers
    """
    try:
        all_growth_data = []
        sectors_with_data = []
        ticker_annotations = []  # Store ticker symbols for each cell

        for sector in SECTORS:
            if sector in sector_data and years in sector_data[sector]:
                df = sector_data[sector][years]
                if not df.empty:
                    # Sort by growth and take top performers
                    df = df.sort_values('Growth', ascending=False).head(10)
                    all_growth_data.append(df['Growth'].values)
                    sectors_with_data.append(sector)
                    # Store ticker symbols
                    ticker_annotations.append(df['Ticker'].values)

        if not all_growth_data:
            return None

        plt.figure(figsize=(15, 8))
        ax = sns.heatmap(
            all_growth_data,
            xticklabels=[f'Top {i+1}' for i in range(10)],
            yticklabels=sectors_with_data,
            cmap='RdYlBu',
            center=0,
            annot=True,
            fmt='.1f',
            cbar_kws={'label': 'Growth %'}
        )

        # Add ticker symbols as text annotations
        for i in range(len(sectors_with_data)):
            for j in range(len(ticker_annotations[i])):
                growth_val = all_growth_data[i][j]
                ticker = ticker_annotations[i][j]
                # Position text below the growth value
                ax.text(j + 0.5, i + 0.7, f'({ticker})',
                       ha='center', va='center',
                       color='black' if abs(growth_val) < 30 else 'white',
                       fontsize=8)

        plt.title(f"{years} Year Growth Performance Heatmap")
        plt.tight_layout()

        img_stream = io.BytesIO()
        plt.savefig(img_stream, format='png', bbox_inches='tight', dpi=300)
        img_stream.seek(0)
        plt.close()

        return Image(img_stream, width=11*inch, height=6*inch)

    except Exception as e:
        print(f"Error generating heatmap: {str(e)}")
        return None

In [105]:
def generate_overall_performance_summary(sector_data, time_periods):
    """
    Generate overall top and bottom performers across all sectors
    """
    try:
        all_stocks = []

        # Collect all stock data across sectors and time periods
        for sector in sector_data:
            for years in time_periods:
                if years in sector_data[sector]:
                    df = sector_data[sector][years]
                    if not df.empty:
                        # Add sector and time period information
                        df = df.copy()  # Create copy to avoid warnings
                        df['Sector'] = sector
                        df['Time Period'] = f"{years} Year"
                        all_stocks.append(df)

        if all_stocks:
            # Combine all stock data
            combined_df = pd.concat(all_stocks, ignore_index=True)

            # Sort and get top/bottom performers
            top_5 = combined_df.nlargest(5, 'Growth')[['Ticker', 'Company', 'Sector', 'Time Period', 'Growth', 'Initial Price', 'Final Price']]
            bottom_5 = combined_df.nsmallest(5, 'Growth')[['Ticker', 'Company', 'Sector', 'Time Period', 'Growth', 'Initial Price', 'Final Price']]

            return top_5, bottom_5

        return None, None
    except Exception as e:
        print(f"Error in performance summary generation: {str(e)}")
        return None, None

def generate_consolidated_report():
    """
    Generate consolidated PDF report with enhanced executive summary
    """
    try:
        print("Starting enhanced PDF report generation...")
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(
            buffer,
            pagesize=landscape(letter),
            rightMargin=36,
            leftMargin=36,
            topMargin=36,
            bottomMargin=36
        )

        styles = getSampleStyleSheet()
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            alignment=1  # Center alignment
        )
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=18,
            spaceAfter=12
        )
        subheading_style = ParagraphStyle(
            'CustomSubHeading',
            parent=styles['Heading3'],
            fontSize=14,
            spaceAfter=8
        )
        normal_style = styles['Normal']

        story = []

        # Title page
        story.append(Paragraph("Stock Market Performance Analysis", title_style))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", normal_style))
        story.append(Spacer(1, 20))

        # Process data for all time periods
        time_periods = [1, 2, 3]
        sector_data = {}

        print("Gathering performance data...")
        for sector in SECTORS:
            sector_data[sector] = {}
            for years in time_periods:
                df = get_stock_performance(sector, years)
                if not df.empty:
                    sector_data[sector][years] = df

        # Executive Summary
        story.append(Paragraph("Executive Summary", heading_style))
        story.append(Spacer(1, 10))

        # Overall top and bottom performers
        print("Generating overall performance summary...")
        top_5, bottom_5 = generate_overall_performance_summary(sector_data, time_periods)

        if top_5 is not None and bottom_5 is not None:
            # Top 5 Performers Table
            story.append(Paragraph("Top 5 Performers Across All Sectors and Time Periods", subheading_style))
            top_data = [['Rank', 'Ticker', 'Company', 'Sector', 'Time Period', 'Growth (%)', 'Initial ($)', 'Final ($)']]

            for idx, row in top_5.iterrows():
                top_data.append([
                    str(len(top_data)),  # Rank
                    row['Ticker'],
                    row['Company'],
                    row['Sector'],
                    row['Time Period'],
                    f"{row['Growth']:.2f}",
                    f"{row['Initial Price']:.2f}",
                    f"{row['Final Price']:.2f}"
                ])

            top_table = Table(top_data, repeatRows=1)
            top_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('FONTSIZE', (0, 1), (-1, -1), 9),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.whitesmoke, colors.white])
            ]))
            story.append(top_table)
            story.append(Spacer(1, 20))

            # Bottom 5 Performers Table
            story.append(Paragraph("Bottom 5 Performers Across All Sectors and Time Periods", subheading_style))
            bottom_data = [['Rank', 'Ticker', 'Company', 'Sector', 'Time Period', 'Growth (%)', 'Initial ($)', 'Final ($)']]

            for idx, row in bottom_5.iterrows():
                bottom_data.append([
                    str(len(bottom_data)),  # Rank
                    row['Ticker'],
                    row['Company'],
                    row['Sector'],
                    row['Time Period'],
                    f"{row['Growth']:.2f}",
                    f"{row['Initial Price']:.2f}",
                    f"{row['Final Price']:.2f}"
                ])

            bottom_table = Table(bottom_data, repeatRows=1)
            bottom_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkred),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('FONTSIZE', (0, 1), (-1, -1), 9),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.mistyrose, colors.white])
            ]))
            story.append(bottom_table)
            story.append(Spacer(1, 20))

        story.append(PageBreak())

        # Performance Heatmaps
        print("Generating performance heatmaps...")
        story.append(Paragraph("Performance Overview by Time Period", heading_style))
        story.append(Spacer(1, 10))

        for years in time_periods:
            heatmap = generate_sector_heatmap(sector_data, years)
            if heatmap:
                story.append(Paragraph(f"{years} Year Performance Analysis", subheading_style))
                story.append(heatmap)
                story.append(Spacer(1, 20))

        story.append(PageBreak())

        # Detailed Sector Analysis
        print("Generating detailed sector analysis...")
        story.append(Paragraph("Detailed Sector Analysis", heading_style))

        for sector in SECTORS:
            story.append(Paragraph(f"{sector} Performance Analysis", subheading_style))

            for years in time_periods:
                if years in sector_data[sector]:
                    df = sector_data[sector][years]
                    if not df.empty:
                        # Generate bar chart
                        chart = generate_bar_chart(df.sort_values('Growth', ascending=True).tail(10),
                                                 sector, years)
                        if chart:
                            story.append(chart)
                            story.append(Spacer(1, 10))

            story.append(Spacer(1, 20))
            story.append(PageBreak())

        print("Building final PDF...")
        doc.build(story)
        buffer.seek(0)
        print("PDF generation complete!")
        return buffer

    except Exception as e:
        print(f"Error in PDF generation: {str(e)}")
        raise

In [106]:
def get_stocks_by_performance_threshold(sector_data, threshold, time_periods, increase=True):
    """
    Extract stocks that meet the performance threshold criteria
    threshold: percentage value
    increase: True for stocks above threshold, False for stocks below threshold
    """
    try:
        filtered_stocks = []

        for sector in sector_data:
            for years in time_periods:
                if years in sector_data[sector]:
                    df = sector_data[sector][years].copy()
                    if not df.empty:
                        df['Sector'] = sector
                        df['Time Period'] = f"{years} Year"

                        # Filter based on threshold
                        if increase:
                            mask = df['Growth'] >= threshold
                        else:
                            mask = df['Growth'] <= -threshold

                        filtered_df = df[mask]
                        if not filtered_df.empty:
                            filtered_stocks.append(filtered_df)

        if filtered_stocks:
            return pd.concat(filtered_stocks, ignore_index=True)
        return pd.DataFrame()

    except Exception as e:
        print(f"Error in threshold filtering: {str(e)}")
        return pd.DataFrame()

def generate_threshold_based_report(threshold=20):
    """
    Generate report focusing on stocks meeting specified threshold criteria
    """
    try:
        print(f"Starting threshold-based report generation (threshold: {threshold}%)...")
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(
            buffer,
            pagesize=landscape(letter),
            rightMargin=36,
            leftMargin=36,
            topMargin=36,
            bottomMargin=36
        )

        # [Previous styles setup remains the same...]
        styles = getSampleStyleSheet()
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            alignment=1
        )
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=18,
            spaceAfter=12
        )
        subheading_style = ParagraphStyle(
            'CustomSubHeading',
            parent=styles['Heading3'],
            fontSize=14,
            spaceAfter=8
        )
        normal_style = styles['Normal']

        story = []

        # Title page
        story.append(Paragraph(f"Stock Performance Analysis (±{threshold}% Threshold)", title_style))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", normal_style))
        story.append(Spacer(1, 20))

        # Process data
        time_periods = [1, 2, 3]
        sector_data = {}

        print("Gathering performance data...")
        for sector in SECTORS:
            sector_data[sector] = {}
            for years in time_periods:
                df = get_stock_performance(sector, years)
                if not df.empty:
                    sector_data[sector][years] = df

        # Get stocks meeting threshold criteria
        outperformers = get_stocks_by_performance_threshold(sector_data, threshold, time_periods, increase=True)
        underperformers = get_stocks_by_performance_threshold(sector_data, threshold, time_periods, increase=False)

        # Executive Summary
        story.append(Paragraph("Executive Summary", heading_style))
        story.append(Spacer(1, 10))

        # Summary statistics
        stats_text = [
            f"Total stocks analyzed: {sum(len(df) for sector in sector_data.values() for df in sector.values())}",
            f"Stocks with ≥{threshold}% growth: {len(outperformers)}",
            f"Stocks with ≤-{threshold}% growth: {len(underperformers)}"
        ]
        for text in stats_text:
            story.append(Paragraph(text, normal_style))
        story.append(Spacer(1, 20))

        # Outperformers Table
        if not outperformers.empty:
            story.append(Paragraph(f"Stocks with {threshold}% or Higher Growth", subheading_style))
            outperformer_data = [['Ticker', 'Company', 'Sector', 'Time Period', 'Growth (%)', 'Initial ($)', 'Final ($)']]

            for _, row in outperformers.sort_values('Growth', ascending=False).iterrows():
                outperformer_data.append([
                    row['Ticker'],
                    row['Company'],
                    row['Sector'],
                    row['Time Period'],
                    f"{row['Growth']:.2f}",
                    f"{row['Initial Price']:.2f}",
                    f"{row['Final Price']:.2f}"
                ])

            out_table = Table(outperformer_data, repeatRows=1)
            out_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkgreen),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('FONTSIZE', (0, 1), (-1, -1), 9),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.lightgreen, colors.white])
            ]))
            story.append(out_table)
            story.append(Spacer(1, 20))

        # Underperformers Table
        if not underperformers.empty:
            story.append(Paragraph(f"Stocks with -{threshold}% or Lower Growth", subheading_style))
            underperformer_data = [['Ticker', 'Company', 'Sector', 'Time Period', 'Growth (%)', 'Initial ($)', 'Final ($)']]

            for _, row in underperformers.sort_values('Growth').iterrows():
                underperformer_data.append([
                    row['Ticker'],
                    row['Company'],
                    row['Sector'],
                    row['Time Period'],
                    f"{row['Growth']:.2f}",
                    f"{row['Initial Price']:.2f}",
                    f"{row['Final Price']:.2f}"
                ])

            under_table = Table(underperformer_data, repeatRows=1)
            under_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkred),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('FONTSIZE', (0, 1), (-1, -1), 9),
                ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.mistyrose, colors.white])
            ]))
            story.append(under_table)
            story.append(Spacer(1, 20))

        story.append(PageBreak())

        # Sector-wise Analysis
        story.append(Paragraph("Sector-wise Distribution", heading_style))

        # Create sector distribution summary
        if not outperformers.empty or not underperformers.empty:
            sector_summary = []
            for sector in SECTORS:
                outperform_count = len(outperformers[outperformers['Sector'] == sector])
                underperform_count = len(underperformers[underperformers['Sector'] == sector])

                sector_summary.append([
                    sector,
                    outperform_count,
                    underperform_count,
                    outperform_count + underperform_count
                ])

            summary_table = Table([
                ['Sector', f'≥{threshold}%', f'≤-{threshold}%', 'Total'],
                *sector_summary
            ])
            summary_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('GRID', (0, 0), (-1, -1), 1, colors.black)
            ]))
            story.append(summary_table)

        # Add visualization if needed
        # [Additional visualizations can be added here]

        print("Building final PDF...")
        doc.build(story)
        buffer.seek(0)
        print("PDF generation complete!")
        return buffer

    except Exception as e:
        print(f"Error in threshold-based report generation: {str(e)}")
        raise


In [107]:
def generate_comprehensive_report(threshold=40.0):
    """
    Generate comprehensive PDF report with visualizations and properly formatted numbers
    """
    try:
        print("Starting comprehensive report generation...")
        buffer = io.BytesIO()

        # Create the PDF document with explicit page size
        width, height = landscape(letter)
        doc = SimpleDocTemplate(
            buffer,
            pagesize=(width, height),
            rightMargin=36,
            leftMargin=36,
            topMargin=36,
            bottomMargin=36
        )

        styles = getSampleStyleSheet()
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            alignment=1
        )
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=18,
            spaceAfter=12
        )
        normal_style = styles['Normal']

        story = []

        # Title
        story.append(Paragraph("Stock Market Performance Analysis", title_style))
        story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", normal_style))
        story.append(Spacer(1, 20))

        # Get all performance data first
        print("Collecting performance data...")
        all_data = []
        sector_data = {}  # For heatmaps

        for sector in SECTORS:
            sector_data[sector] = {}
            for years in [1, 2, 3]:
                df = get_stock_performance(sector, years)
                if not df.empty:
                    df['Sector'] = sector
                    df['Time Period'] = f"{years} Year"
                    all_data.append(df)
                    sector_data[sector][years] = df

        if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)
            outperformers = combined_df[combined_df['Growth'] >= threshold]
            underperformers = combined_df[combined_df['Growth'] <= -threshold]

            # Summary Statistics
            story.append(Paragraph("Performance Summary", heading_style))
            story.append(Paragraph(f"Total stocks analyzed: {len(combined_df['Ticker'].unique())}", normal_style))
            story.append(Paragraph(f"Stocks with ≥{threshold:,.1f}% growth: {len(outperformers)}", normal_style))
            story.append(Paragraph(f"Stocks with ≤-{threshold:,.1f}% growth: {len(underperformers)}", normal_style))
            story.append(Spacer(1, 20))

            # Top Performers Table
            if not outperformers.empty:
                story.append(Paragraph(f"Top Performers (≥{threshold:,.1f}%)", heading_style))
                top_data = [['Ticker', 'Company', 'Sector', 'Period', 'Growth (%)', 'Initial ($)', 'Final ($)']]

                for _, row in outperformers.sort_values('Growth', ascending=False).iterrows():
                    top_data.append([
                        row['Ticker'],
                        row['Company'][:30],  # Limit company name length
                        row['Sector'],
                        row['Time Period'],
                        f"{row['Growth']:,.1f}",
                        f"{row['Initial Price']:,.2f}",
                        f"{row['Final Price']:,.2f}"
                    ])

                table = Table(top_data, repeatRows=1)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.darkgreen),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 10),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ]))
                story.append(table)
                story.append(Spacer(1, 20))

            story.append(PageBreak())

            # Performance Overview with Heatmaps
            story.append(Paragraph("Performance Overview by Time Period", heading_style))
            story.append(Spacer(1, 10))

            for years in [1, 2, 3]:
                story.append(Paragraph(f"{years} Year Performance Analysis", heading_style))
                # Add heatmap
                heatmap = generate_sector_heatmap(sector_data, years)
                if heatmap:
                    story.append(heatmap)
                story.append(Spacer(1, 20))

            story.append(PageBreak())

            # Detailed Sector Analysis with Bar Charts
            story.append(Paragraph("Detailed Sector Analysis", heading_style))

            for sector in SECTORS:
                story.append(Paragraph(f"{sector} Performance Analysis", heading_style))
                sector_df = combined_df[combined_df['Sector'] == sector]

                for years in [1, 2, 3]:
                    period_df = sector_df[sector_df['Time Period'] == f"{years} Year"]
                    if not period_df.empty:
                        # Add bar chart
                        chart = generate_bar_chart(
                            period_df.sort_values('Growth', ascending=True).tail(10),
                            sector,
                            years
                        )
                        if chart:
                            story.append(chart)

                        # Add performance table
                        data = [['Ticker', 'Growth (%)', 'Initial ($)', 'Final ($)']]
                        for _, row in period_df.sort_values('Growth', ascending=False).iterrows():
                            data.append([
                                row['Ticker'],
                                f"{row['Growth']:,.1f}",
                                f"{row['Initial Price']:,.2f}",
                                f"{row['Final Price']:,.2f}"
                            ])

                        table = Table(data, repeatRows=1)
                        table.setStyle(TableStyle([
                            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                            ('GRID', (0, 0), (-1, -1), 1, colors.black)
                        ]))
                        story.append(Paragraph(f"{years} Year Performance", normal_style))
                        story.append(table)
                        story.append(Spacer(1, 10))

                story.append(PageBreak())

        # Build PDF
        doc.build(story)
        buffer.seek(0)
        return buffer

    except Exception as e:
        print(f"Error generating PDF: {str(e)}")
        raise

In [108]:
# App layout
app.layout = html.Div(
    style={
        'backgroundColor': 'black',
        'color': 'white',
        'padding': '20px',
        'minHeight': '100vh'
    },
    children=[
        html.H1('Stock Performance Analyzer',
                style={'textAlign': 'center', 'color': 'white', 'marginBottom': '30px'}),

        # Controls Section
        html.Div([
            # First Row - Sector Dropdown and Threshold Input
            html.Div([
                # Sector Dropdown
                html.Div([
                    html.Label('Select Industry Sector:',
                              style={'color': 'white', 'marginRight': '10px', 'fontSize': '16px'}),
                    dcc.Dropdown(
                        id='sector-dropdown',
                        options=[{'label': sector, 'value': sector} for sector in SECTORS],
                        value=SECTORS[0] if SECTORS else None,
                        style={
                            'width': '300px',
                            'backgroundColor': 'white',
                            'color': 'black',
                        }
                    ),
                ], style={'display': 'flex', 'alignItems': 'center', 'marginRight': '40px'}),

                # Performance Threshold Input
                html.Div([
                    html.Label('Performance Threshold (%):',
                              style={'color': 'white', 'marginRight': '10px', 'fontSize': '16px'}),
                    dcc.Input(
                        id='threshold-input',
                        type='number',
                        value=20,
                        min=0,
                        max=100,
                        step=5,
                        style={
                            'width': '100px',
                            'height': '36px',
                            'padding': '5px',
                            'borderRadius': '4px',
                            'border': '1px solid #ccc'
                        }
                    ),
                ], style={'display': 'flex', 'alignItems': 'center'})
            ], style={'display': 'flex', 'marginBottom': '20px'}),

            # Second Row - Report Generation and Status
            html.Div([
                html.Button(
                    'Generate Report (PDF)',
                    id='generate-button',
                    style={
                        'padding': '10px 20px',
                        'backgroundColor': '#4CAF50',
                        'color': 'white',
                        'border': 'none',
                        'borderRadius': '5px',
                        'cursor': 'pointer',
                        'marginRight': '10px',
                        'fontSize': '14px'
                    }
                ),
                html.Div(
                    id='download-status',
                    style={
                        'marginLeft': '10px',
                        'color': '#4CAF50',
                        'display': 'inline-block',
                        'fontSize': '14px'
                    }
                ),
                dcc.Loading(
                    id="loading",
                    type="circle",
                    children=[html.Div(id="loading-output")],
                    style={'marginLeft': '10px'}
                ),
                dcc.Download(id='download-pdf')
            ], style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '20px'}),
        ]),

        # Time Period Tabs
        dcc.Tabs(
            id='timeframe-tabs',
            value='1year',
            children=[
                dcc.Tab(
                    label='1 Year',
                    value='1year',
                    style={
                        'backgroundColor': 'black',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    },
                    selected_style={
                        'backgroundColor': 'grey',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    }
                ),
                dcc.Tab(
                    label='2 Years',
                    value='2years',
                    style={
                        'backgroundColor': 'black',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    },
                    selected_style={
                        'backgroundColor': 'grey',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    }
                ),
                dcc.Tab(
                    label='3 Years',
                    value='3years',
                    style={
                        'backgroundColor': 'black',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    },
                    selected_style={
                        'backgroundColor': 'grey',
                        'color': 'white',
                        'padding': '10px 20px',
                        'borderRadius': '5px 5px 0 0'
                    }
                )
            ],
            style={
                'marginBottom': '20px',
                'borderBottom': '1px solid grey'
            }
        ),

        # Visualization Area
        dcc.Graph(
            id='performance-graph',
            style={
                'backgroundColor': 'black',
                'marginBottom': '20px',
                'border': '1px solid grey',
                'borderRadius': '5px'
            }
        ),

        # Details Table
        html.Div(
            id='stock-details',
            style={
                'marginTop': '20px',
                'color': 'white',
                'padding': '20px',
                'border': '1px solid grey',
                'borderRadius': '5px'
            }
        )
    ]
)

# Callbacks
@app.callback(
    [Output('performance-graph', 'figure'),
     Output('stock-details', 'children')],
    [Input('sector-dropdown', 'value'),
     Input('timeframe-tabs', 'value')]
)
def update_graph(selected_sector, selected_timeframe):
    empty_fig = px.bar(title="No data available")
    empty_fig.update_layout(
        plot_bgcolor='black',
        paper_bgcolor='black',
        font_color='white'
    )

    if not selected_sector:
        return empty_fig, "Please select a sector"

    try:
        print(f"Fetching data for {selected_sector} over {selected_timeframe}")
        years = int(selected_timeframe[0])
        df = get_stock_performance(selected_sector, years)

        if df.empty:
            return empty_fig, f"No data available for {selected_sector}"

        df = df.sort_values('Growth', ascending=True).tail(10)

        fig = px.bar(
            df,
            x='Growth',
            y='Ticker',
            orientation='h',
            color='Ticker',
            title=f'Top 10 Performing Stocks in {selected_sector} ({years} Year Growth)',
            hover_data=['Company']
        )

        fig.update_layout(
            plot_bgcolor='black',
            paper_bgcolor='black',
            font_color='white',
            title_font_color='white',
            showlegend=True,
            xaxis_title='Growth (%)',
            yaxis_title='Stock Ticker',
            xaxis=dict(
                gridcolor='gray',
                showgrid=True,
                zeroline=True,
                zerolinecolor='gray'
            ),
            yaxis=dict(
                gridcolor='gray',
                showgrid=True
            ),
            margin=dict(l=50, r=50, t=50, b=50),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            )
        )

        details = html.Div([
            html.H3('Performance Details', style={'marginBottom': '15px'}),
            html.Table(
                [html.Tr([
                    html.Th(col, style={'padding': '10px', 'borderBottom': '1px solid white'})
                    for col in ['Ticker', 'Company', 'Growth (%)', 'Initial Price ($)', 'Final Price ($)']
                ])] + [
                    html.Tr([
                        html.Td(row['Ticker'], style={'padding': '10px'}),
                        html.Td(row['Company'], style={'padding': '10px'}),
                        html.Td(f"{row['Growth']:.2f}", style={'padding': '10px'}),
                        html.Td(f"{row['Initial Price']:.2f}", style={'padding': '10px'}),
                        html.Td(f"{row['Final Price']:.2f}", style={'padding': '10px'})
                    ], style={'borderBottom': '1px solid #333'})
                    for _, row in df.iterrows()
                ],
                style={
                    'width': '100%',
                    'borderCollapse': 'collapse',
                    'marginTop': '10px'
                }
            )
        ])

        return fig, details

    except Exception as e:
        print(f"Error updating graph: {str(e)}")
        return empty_fig, f"Error loading data: {str(e)}"

# Make sure to update the callback accordingly:
@app.callback(
    [Output('download-pdf', 'data'),
     Output('download-status', 'children', allow_duplicate=True),
     Output('loading-output', 'children', allow_duplicate=True)],
    [Input('generate-button', 'n_clicks'),
     Input('threshold-input', 'value')],
    prevent_initial_call=True
)
def handle_pdf_generation(n_clicks, threshold_value):
    if n_clicks:
        try:
            print("Starting PDF generation process...")
            threshold = float(threshold_value) if threshold_value else 40.0
            pdf_buffer = generate_comprehensive_report(threshold)

            if pdf_buffer:
                return (
                    dcc.send_bytes(pdf_buffer.read(), 'stock_performance_report.pdf'),
                    "✓ Report downloaded successfully!",
                    ""
                )
            else:
                return None, "❌ Error: PDF generation failed", ""
        except Exception as e:
            print(f"Detailed error in PDF generation: {str(e)}")
            return None, f"❌ Error: {str(e)}", ""
    return None, "", ""

@app.callback(
    [Output('download-status', 'children'),
     Output('loading-output', 'children')],
    [Input('sector-dropdown', 'value')],
    prevent_initial_call=True
)
def initialize_status(value):
    return "", ""

In [109]:
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port=8050)

<IPython.core.display.Javascript object>